### Setup domain:

notebooks/Experimental/Ishan/AA/setup-data.ipynb

In [1]:
%load_ext autoreload
%autoreload 2
import syft as sy
from tqdm import tqdm
import time

In [2]:
domain_addresses = [8081, 8082]  # replace with URLs
username = "info@openmined.org"  # "sam@stargate.net"
pw = "changethis"

In [3]:
from syft import nn

def initialize_model() -> nn.Model:
    model = nn.Model()

    # Layer 1
    model.add(nn.Convolution(nb_filter=32, filter_size=3, padding=2, input_shape=input_shape))
    model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
    model.add(nn.MaxPool(pool_size=2, stride=2))

    # Layer 2
    model.add(nn.Convolution(nb_filter=64, filter_size=3, padding=2))
    model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
    model.add(nn.MaxPool(pool_size=2, stride=2))

    # # Layer 3
    # model.add(nn.Convolution(nb_filter=128, filter_size=3, padding=2))
    # model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
    # model.add(nn.MaxPool(pool_size=2, stride=2))

    # # Layer 4
    # model.add(nn.Convolution(nb_filter=256, filter_size=3, padding=2))
    # model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
    # model.add(nn.MaxPool(pool_size=2, stride=2))

    # Layer 5
    model.add(nn.AvgPool(2))

    # Layer 6
    model.add(nn.Flatten())

    # Layer 7
    model.add(nn.Linear(2, 512))

    model.initialize_weights()
    return model

In [4]:
# TRAINING PARAMS
n_epochs = 1
batch_size = 2
input_shape = (2,3,50,50)
model = initialize_model()

for address in tqdm(domain_addresses):
    domain = sy.login(email=username, password=pw, port=address)  # change to URL when running on 100
    
    if len(domain.datasets) == 0:
        print(f"Error on domain = {domain.name} with address = {address}")
    
    X_train = domain.datasets[-1]["train_images"][:2]
    y_train = domain.datasets[-1]["train_labels"][:2]
    input_shape = X_train.public_shape
    
    # Do a single epoch
    model_ptr = model.send(domain,send_to_blob_storage=False)
    run_status = model_ptr.step(X_train, y_train)
    while(not run_status.exists):
        time.sleep(10)
    
    # Publish Model Weights
    published_obj = model_ptr.publish(sigma=1e4)  # PB spent with 1e3 = 690k, thus 1e4 sigma -> 6.9k which is within PB limit of 9999
    while(not published_obj.exists):
        time.sleep(2)
    new_weights = published_obj.get_copy()
    
    # Update weights and move onto next domain node
    model = initialize_model()  # replacing weights into existing model causes a NotImplementedError
    model.replace_weights(new_weights)    

  0%|                                                                                                                           | 0/2 [00:00<?, ?it/s]


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into canada... done!


  0%|                                                                                                                           | 0/2 [06:19<?, ?it/s]

['Convolution0', 'BatchNorm1', 'MaxPool2', 'Convolution3', 'BatchNorm4', 'MaxPool5', 'AvgPool6', 'Flatten7', 'Linear8']


TypeError: '>' not supported between instances of 'list' and 'int'

In [5]:
new_model = model.replace_weights(new_weights)

['Convolution0', 'BatchNorm1', 'MaxPool2', 'Convolution3', 'BatchNorm4', 'MaxPool5', 'AvgPool6', 'Flatten7', 'Linear8']


In [6]:
new_model